03/29/24 JD

Goal: `workerScope4A_WGL.py` is the parallel version of running the entire dataset. In this jupyter notebook, we will explore generating fits data for `FOV=3` for three rounds of imaging (`H1`, `H10`, `H60`). 

H1 (comb) and H10 (seq), H60 (ctrl).

Note: control only has spots in Cy5 and Cy3


Reference file: `/mnt/tscc/wed009/WholeGenome/fits_scripts/NMERFISH_Jenny/workers/workerScope4A_WGL.py`

In [1]:
from ioMicro import *

In [2]:
import time,sys
import os,sys,numpy as np

This is the main worker function #L150:

```def compute_main_f(save_folder,all_flds,fov,ifov,redo_fits,redo_drift,try_mode,old_method):
    print("Computing fitting on: "+str(fov))
    print(len(all_flds),all_flds)
    compute_fits(save_folder,fov,all_flds,redo=redo_fits,try_mode=try_mode,old_method=old_method)
    print("Computing drift on: "+str(fov))
    compute_drift_features(save_folder,fov,all_flds,redo=False,gpu=True)
    compute_drift_V2(save_folder,fov,all_flds,redo=redo_drift,gpu=True)```

In [3]:
master_analysis_folder = '/mnt/tscc/wed009/WholeGenome'

### Did you compute PSF and median flat field images?
psf_file = '/mnt/tscc/wed009/WholeGenome/fits_scripts/NMERFISH/psfs/psf_750_Scope4_final.npy'
flat_field_tag = '/mnt/tscc/wed009/WholeGenome/fits_scripts/NMERFISH/flat_field/Scope4_'#med_col_rawXXX.npz

master_data_folder = ['/projects/ps-renlab2/wed009/031524_v1_acry_37C_pilot/']
save_folder ='/projects/ps-renlab2/wed009/analysis_031524_v1_acry_37C_pilot/'

# Get Files

In [4]:
def get_iH(fld): 
    """
    #L64
    Given a folder <fld> of type 
    /projects/ps-renlab2/wed009/H1_P1_A1_2_3/
    this extracts the hybe index (i.e. 2 in this case)
    """
    try:
        return int(os.path.basename(fld).split('_')[0][1:]) #TODO: modified for WGL
    except:
        return np.inf

def get_files(ifov):
    '''
    #L73
    '''
    if not os.path.exists(save_folder): os.makedirs(save_folder)
    all_flds = []
    for master_folder in master_data_folder:
        all_flds += glob.glob(master_folder+r'/H*') # TODO: modified for WGL
        
    ### reorder based on hybe index
    all_flds = np.array(all_flds)[np.argsort([get_iH(fld)for fld in all_flds])] 
    
    ### find all the fovs
    fovs_fl = save_folder+os.sep+'fovs__.npy'
    
    if not os.path.exists(fovs_fl):
        folder_map_fovs = all_flds[-1]
        fls = glob.glob(folder_map_fovs+os.sep+'*.zarr')
        fovs = np.sort([os.path.basename(fl) for fl in fls])
        np.save(fovs_fl,fovs)
    else:
        fovs = np.sort(np.load(fovs_fl))
    fov=None
    if ifov<len(fovs):
        fov = fovs[ifov]
        all_flds = [fld for fld in all_flds if os.path.exists(fld+os.sep+fov)]
    # but only return the ifov (with formatted filename)   
    return save_folder,all_flds,fov

In [5]:
save_folder,all_flds,fov = get_files(ifov = 3)

In [6]:
all_flds
len(all_flds) # total should be 68 rounds of imaging

68

In [8]:
chrom_abb_rounds = ['H65', 'H66', 'H67']
chrom_abb_flds = [s for s in all_flds if any(sub in s for sub in chrom_abb_rounds)]
chrom_abb_flds

['/projects/ps-renlab2/wed009/031524_v1_acry_37C_pilot/H65_P2_F10_11_12',
 '/projects/ps-renlab2/wed009/031524_v1_acry_37C_pilot/H66_P2_F10_11_12_750',
 '/projects/ps-renlab2/wed009/031524_v1_acry_37C_pilot/H67_P2_F10_11_12_cy5']

In [15]:
# test_flds only has H1 (comb) and H10 (seq), H60 (ctrl)
test_flds = [all_flds[i] for i in [0,9,59] if i < len(all_flds)]
test_flds

['/projects/ps-renlab2/wed009/031524_v1_acry_37C_pilot/H1_P1_A1_2_3',
 '/projects/ps-renlab2/wed009/031524_v1_acry_37C_pilot/H10_P1_C4_5_6',
 '/projects/ps-renlab2/wed009/031524_v1_acry_37C_pilot/H60_P3_C4_5_6']

In [16]:
fov

'Conv_zscan__03.zarr'

In [10]:
for i in range(1, 68):
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67


# Compute Fits

In [19]:
def compute_fits(save_folder,fov,all_flds,redo=False,ncols=4,
                psf_file = psf_file,try_mode=True,old_method=False):
    """This wraps main_do_compute_fits to allow for try_mode"""
    psf = np.load(psf_file)
    
    for fld in tqdm(all_flds):
        for icol in range(ncols-1):
            tag = os.path.basename(fld)
            save_fl = save_folder+os.sep+fov.split('.')[0]+'--'+tag+'--col'+str(icol)+'__Xhfits.npz'
            try:
                np.load(save_fl)['Xh']
                redo2 = False
            except:
                redo2 = True
            if not os.path.exists(save_fl) or redo or redo2:
                if try_mode:
                    try:
                        main_do_compute_fits(save_folder,fld,fov,icol,save_fl,psf,old_method)
                    except:
                        print("Failed",fld,fov,icol)
                else:
                    main_do_compute_fits(save_folder,fld,fov,icol,save_fl,psf,old_method)            

def main_do_compute_fits(save_folder,fld,fov,icol,save_fl,psf,old_method):
    im_ = read_im(fld+os.sep+fov)
    im__ = np.array(im_[icol],dtype=np.float32)
    
    if old_method:
        ### previous method - no deconvolution
        im_n = norm_slice(im__,s=30)
        Xh = get_local_maxfast_tensor(im_n,th_fit=500,im_raw=im__,dic_psf=None,delta=1,delta_fit=3,sigmaZ=1,sigmaXY=1.5,gpu=False)
    else:
        ### new method - using deconvolution
        fl_med = flat_field_tag+'med_col_raw'+str(icol)+'.npz'

        im_med = np.array(np.load(fl_med)['im'],dtype=np.float32)
        im_med = cv2.blur(im_med,(20,20))
        im__ = im__/im_med*np.median(im_med)


        Xh = get_local_max_tile(im__,th=3600,s_ = 500,pad=100,psf=psf,plt_val=None,snorm=30,gpu=True,
                                deconv={'method':'wiener','beta':1e-4}, #TODO: original beta = 0.0001 (1e-4)
                                delta=1,delta_fit=3,sigmaZ=1,sigmaXY=1.5)
    np.savez_compressed(save_fl,Xh=Xh)

In [20]:
# first 3 params calculated by get_files(ifov = 3)
# only do it on test_flds
compute_fits(save_folder = save_folder,
             fov = fov,
             all_flds = test_flds,
             redo=True,ncols=4,psf_file = psf_file,try_mode=True,old_method=False)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [02:45<00:00, 55.26s/it]


`norm_slice` back ground subtraction

change `th = 1000`

## load fits result

In [25]:
print('H1, comb')
print(f"A750: {len(np.load(os.path.join(save_folder, 'Conv_zscan__03--H1_P1_A1_2_3--col0__Xhfits.npz'))['Xh'])}")
print(f"Cy5: {len(np.load(os.path.join(save_folder, 'Conv_zscan__03--H1_P1_A1_2_3--col1__Xhfits.npz'))['Xh'])}")
print(f"Cy3: {len(np.load(os.path.join(save_folder, 'Conv_zscan__03--H1_P1_A1_2_3--col2__Xhfits.npz'))['Xh'])}")

H1, comb
A750: 150867
Cy5: 175648
Cy3: 11666


In [26]:
print('H10, seq')
print(f"A750: {len(np.load(os.path.join(save_folder, 'Conv_zscan__03--H10_P1_C4_5_6--col0__Xhfits.npz'))['Xh'])}")
print(f"Cy5: {len(np.load(os.path.join(save_folder, 'Conv_zscan__03--H10_P1_C4_5_6--col1__Xhfits.npz'))['Xh'])}")
print(f"Cy3: {len(np.load(os.path.join(save_folder, 'Conv_zscan__03--H10_P1_C4_5_6--col2__Xhfits.npz'))['Xh'])}")

H10, seq
A750: 214817
Cy5: 167375
Cy3: 9458


In [27]:
print('H60, ctrl') # ctrl should only have colors in Cy5 and Cy3
print(f"A750: {len(np.load(os.path.join(save_folder, 'Conv_zscan__03--H60_P3_C4_5_6--col0__Xhfits.npz'))['Xh'])}")
print(f"Cy5: {len(np.load(os.path.join(save_folder, 'Conv_zscan__03--H60_P3_C4_5_6--col1__Xhfits.npz'))['Xh'])}")
print(f"Cy3: {len(np.load(os.path.join(save_folder, 'Conv_zscan__03--H60_P3_C4_5_6--col2__Xhfits.npz'))['Xh'])}")

H60, ctrl
A750: 34240
Cy5: 100380
Cy3: 17395


# Compute drift features

compute_drift_features(save_folder,fov,all_flds,redo=False,gpu=True) # L155

In [28]:
def compute_drift_features(save_folder,fov,all_flds,redo=False,gpu=True):
    '''
    L102
    '''
    fls = [fld+os.sep+fov for fld in all_flds]
    set_ = ''
    for fl in fls:
        get_dapi_features(fl,save_folder,set_,gpu=gpu,im_med_fl = flat_field_tag+r'med_col_raw3.npz',
                    psf_fl = psf_file) ### devonvolve the dapi image and fit local minima and maxima (dapi features)

In [29]:
# first 3 param computed by get_files()
# do it for all FOV
compute_drift_features(save_folder = save_folder,
                       fov = fov,
                       all_flds = test_flds,
                       redo=False,gpu=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:06<00:00,  5.70it/s]


In [32]:
print(f"H1: {len(np.load(os.path.join(save_folder, 'Conv_zscan__03--H1_P1_A1_2_3--dapiFeatures.npz'))['Xh_plus'])}")
print(f"H1: {len(np.load(os.path.join(save_folder, 'Conv_zscan__03--H10_P1_C4_5_6--dapiFeatures.npz'))['Xh_plus'])}")
print(f"H60: {len(np.load(os.path.join(save_folder, 'Conv_zscan__03--H60_P3_C4_5_6--dapiFeatures.npz'))['Xh_plus'])}")

H1: 24041
H1: 25265
H60: 28194


In [36]:
# first 3 param computed by get_files()
# do it for all FOV
compute_drift_features(save_folder = save_folder,
                       fov = fov,
                       all_flds = all_flds,
                       redo=False,gpu=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:05<00:00,  6.16it/s]


# Get drift corrected points

In [34]:
def compute_drift_V2(save_folder,fov,all_flds,redo=False,gpu=True):
    '''
    L134
    '''
    set_ = ''
    drift_fl = save_folder+os.sep+'driftNew_'+fov.split('.')[0]+'--'+set_+'.pkl'
    try:
        np.load(drift_fl,allow_pickle=True)
    except:
        redo=True
    if not os.path.exists(drift_fl) or redo:
        fls = [fld+os.sep+fov for fld in all_flds]
        fl_ref = fls[len(fls)//2]
        newdrifts = []
        for fl in fls:
            drft = get_best_translation_pointsV2(fl,fl_ref,save_folder,resc=5)
            print(drft)
            newdrifts.append(drft)
        pickle.dump([newdrifts,all_flds,fov,fl_ref],open(drift_fl,'wb'))
        
def get_best_translation_pointsV2(fl,fl_ref,save_folder,resc=5,th=4):
    '''
    L109
    '''
    ### THis loads the dapi features and registers the images for files fl and fl_ref
    set_ = ''
    obj = get_dapi_features(fl,save_folder,set_)
    obj_ref = get_dapi_features(fl_ref,save_folder,set_)
    tzxyf,tzxy_plus,tzxy_minus,N_plus,N_minus = np.array([0,0,0]),np.array([0,0,0]),np.array([0,0,0]),0,0
    if (len(obj.Xh_plus)>0) and (len(obj_ref.Xh_plus)>0):
        X = obj.Xh_plus#[:,:3]
        X_ref = obj_ref.Xh_plus#[:,:3]
        X = X[X[:,-1]>th][:,:3]
        X_ref = X_ref[X_ref[:,-1]>th][:,:3]
        tzxy_plus,N_plus = get_best_translation_points(X,X_ref,resc=resc,return_counts=True)
    if (len(obj.Xh_minus)>0) and (len(obj_ref.Xh_minus)>0):
        X = obj.Xh_minus#[:,:3]
        X_ref = obj_ref.Xh_minus#[:,:3]
        X = X[X[:,-1]>th][:,:3]
        X_ref = X_ref[X_ref[:,-1]>th][:,:3]
        tzxy_minus,N_minus = get_best_translation_points(X,X_ref,resc=resc,return_counts=True)
    if np.max(np.abs(tzxy_minus-tzxy_plus))<=2:
        tzxyf = -(tzxy_plus*N_plus+tzxy_minus*N_minus)/(N_plus+N_minus)
    else:
        tzxyf = -[tzxy_plus,tzxy_minus][np.argmax([N_plus,N_minus])]
    

    return [tzxyf,tzxy_plus,tzxy_minus,N_plus,N_minus]

In [35]:
compute_drift_V2(save_folder = save_folder,
                       fov = fov,
                       all_flds = test_flds,
                     redo=False,gpu=True)

[array([ 0.01697627, 11.263697  , -4.0265684 ], dtype=float32), array([ -0.0233192, -11.249808 ,   4.0549564], dtype=float32), array([-5.6384420e-03, -1.1288523e+01,  3.9758267e+00], dtype=float32), 11245, 6291]
[array([-0., -0., -0.], dtype=float32), array([0., 0., 0.], dtype=float32), array([0., 0., 0.], dtype=float32), 15202, 8879]
[array([-1.0073842e-01, -1.0555382e+02,  6.3297066e+01], dtype=float32), array([  0.1180039, 105.53165  , -63.288513 ], dtype=float32), array([ 6.8208709e-02,  1.0559557e+02, -6.3313179e+01], dtype=float32), 10728, 5694]


# napari

In [ ]:
import napari
napari.view_image(np.load(psf_file))

Cannot move to target thread (0x556d64afa960)



Can't use napari on mediator :( A100 GPU doesnt support that

In the end, switched to FOV = 05 because fov3 has a bright light source on the lower left corner. 